In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 551 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 27.3 MB/s 
  Created wheel for openai: filename=openai-0.19.0-py3-none-any.whl size=53535 sha256=11af833f69e9113df99bc8ee83104f5e85a87d897811309e4660ba74e512ee79
  Stored in directory: /root/.cache/pip/wheels/94/b5/c0/928013bd6418b23b9c5d89fb24cdeb1faae899c11377d69609
Successfully built openai


In [ ]:
import os
import openai
import copy
import scipy.stats
import sklearn.datasets
import pandas as pd
import numpy as np
from scipy.io.arff import loadarff 
import urllib
import re

## Domains

### Load data

#### Load datasets

In [ ]:
datasets_dict = {}
datasets_links = {"City_Zhvi_AllHomes.csv": {"link": "https://drive.google.com/u/0/uc?id=1X6zL5ThNJ4RXghus7t-EisW7EyJbG2_B&export=download", "sep": None},
                  "countries of the world.csv": {"link": "https://drive.google.com/u/0/uc?id=1WhV3fwEs9ncnXNrTXhDObYmvUU3ifanY&export=download", "sep": ";"},
                  "KaggleV2-May-2016.csv": {"link": "https://drive.google.com/u/0/uc?id=1DYACMdfKjdDyYCwOlUINNHEvZ7P_N8tz&export=download", "sep": None},
                  "name_gender_dataset.csv": {"link": "https://drive.google.com/u/0/uc?id=148keFgTs_RIJ1ckcV6mG8nKjKnarYfr0&export=download", "sep": None},
                  "speeddating.csv": {"link": "https://drive.google.com/u/0/uc?id=1FMUtpd8_oMJz5Lomf-mNmvkqJUeQYtrm&export=download", "sep": None}
                  }
metadata_link = "https://drive.google.com/u/0/uc?id=18MlqQ-Ea-HIkmWJkjOXqXDxeCrFp98ej&export=download"

In [ ]:
for elem in datasets_links:
  datasets_dict[elem] = pd.read_csv(datasets_links[elem]["link"], sep = datasets_links[elem]["sep"])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
metadata = pd.read_csv(metadata_link, sep = ";")

In [ ]:
metadata.set_index(["dataset", "colnum"], inplace=True)
metadata_dict_pre = metadata.squeeze().unstack().to_dict("index")

In [ ]:
metadata_dict = copy.deepcopy(metadata_dict_pre)
for dataset in metadata_dict_pre:
  for column in metadata_dict_pre[dataset]:
    if type(metadata_dict_pre[dataset][column]) != str:
      metadata_dict[dataset].pop(column)
    else:
      metadata_dict[dataset][column] = metadata_dict_pre[dataset][column].split(",")

In [ ]:
metadata_dict

{'City_Zhvi_AllHomes.csv': {'col2': ['city'],
  'col3': ['state'],
  'col4': ['metropolitan area'],
  'col5': ['county']},
 'KaggleV2-May-2016.csv': {'col3': ['gender'],
  'col4': ['time', 'date'],
  'col5': ['time', 'date'],
  'col6': ['age'],
  'col7': ['neighbourhood']},
 'countries of the world.csv': {'col1': ['country'], 'col2': ['region']},
 'name_gender_dataset.csv': {'col1': ['name'], 'col2': ['gender']},
 'speeddating.csv': {'col3': ['gender'],
  'col4': ['age'],
  'col8': ['race'],
  'col9': ['race']}}

### Run experiment

In [ ]:
def generate_prompt_1(data):
  prompt = f'What is the best domain that describes the values in {str("{")}{data}{str("}")}?'
  return prompt

def generate_prompt_2(data):
  prompt = f'What is the best semantic type that describes the values in {str("{")}{data}{str("}")}?'
  return prompt

In [ ]:
openai.api_key = "COMPLETE"

In [ ]:
results = {}
prompts = {}
n = 5
sample=10
temp = 0

for dataset in metadata_dict:
  print(f"Dataset: {dataset}")

  results[dataset] = {}
  prompts[dataset] = {}

  for attribute in metadata_dict[dataset]:
    print(f"Attribute: {attribute}")

    results[dataset][attribute] = {}
    prompts[dataset][attribute] = {}
    
    values = datasets_dict[dataset][attribute].unique()

    seed = 0
    for rep in range(0, n):
      np.random.seed(seed)
      sampled_values = np.random.choice(values, sample, replace=True)
      sampled_str = ", ".join([str(elem).rstrip() for elem in sampled_values])

      prompts[dataset][attribute][rep] = {}
      prompt_1 = generate_prompt_1(sampled_str)
      prompts[dataset][attribute][rep]["prompt_1"] = prompt_1
      prompt_2 = generate_prompt_2(sampled_str)
      prompts[dataset][attribute][rep]["prompt_2"] = prompt_2

      results[dataset][attribute][rep] = {}

      response_1 = openai.Completion.create(engine="text-davinci-002", prompt=prompt_1, temperature=temp, max_tokens=1000)
      text_1 = response_1["choices"][0]["text"]
      results[dataset][attribute][rep]["prompt_1"] = text_1
      response_2 = openai.Completion.create(engine="text-davinci-002", prompt=prompt_2, temperature=temp, max_tokens=1000)
      text_2 = response_2["choices"][0]["text"]
      results[dataset][attribute][rep]["prompt_2"] = text_2

      print(prompt_1)
      print(text_1)

      print(prompt_2)
      print(text_2)

      seed += 1

Dataset: City_Zhvi_AllHomes.csv
Attribute: col2
What is the best domain that describes the values in {Palmyra, Defiance, Hunters Creek Village, Caddo Mills, Freedom, Charlotte Hall, Lake Como, Troutman, Valrico, Verona}?


The best domain that describes the values in {Palmyra, Defiance, Hunters Creek Village, Caddo Mills, Freedom, Charlotte Hall, Lake Como, Troutman, Valrico, Verona} is "small towns."
What is the best semantic type that describes the values in {Palmyra, Defiance, Hunters Creek Village, Caddo Mills, Freedom, Charlotte Hall, Lake Como, Troutman, Valrico, Verona}?


The best semantic type that describes the values in {Palmyra, Defiance, Hunters Creek Village, Caddo Mills, Freedom, Charlotte Hall, Lake Como, Troutman, Valrico, Verona} is Place.
What is the best domain that describes the values in {Norman, Frackville, Burke, Ione, River Edge, Madeira Beach, Newport News, Talent, Sugar Valley, Corte Madera}?


The best domain would be "towns in the United States."
What is th

In [ ]:
results

{'City_Zhvi_AllHomes.csv': {'col2': {0: {'prompt_1': '\n\nThe best domain that describes the values in {Palmyra, Defiance, Hunters Creek Village, Caddo Mills, Freedom, Charlotte Hall, Lake Como, Troutman, Valrico, Verona} is "small towns."',
    'prompt_2': '\n\nThe best semantic type that describes the values in {Palmyra, Defiance, Hunters Creek Village, Caddo Mills, Freedom, Charlotte Hall, Lake Como, Troutman, Valrico, Verona} is Place.'},
   1: {'prompt_1': '\n\nThe best domain would be "towns in the United States."',
    'prompt_2': '\n\nThe best semantic type that describes the values in the given set is "cities."'},
   2: {'prompt_1': '\n\nOne possible domain that describes the values in the given set is "towns and cities."',
    'prompt_2': '\n\nThe best semantic type that describes the values in {Bridport, Town of Oxford, Saint Lawrence, Crete, Pacifica, North Franklin Township, Melbourne, Vilonia, Oak Grove, June Park} is Place.'},
   3: {'prompt_1': '\n\nThe best domain that

## Evaluate

In [ ]:
evaluation = {}
for prompt in ["prompt_1", "prompt_2"]:
  print(prompt)
  evaluation[prompt] = {}
  
  for dataset in results:
    print(f"Dataset: {dataset}\n")
    evaluation[prompt][dataset] = {}
    evaluation[prompt][dataset]["total_hits"] = 0

    for attribute in results[dataset]:

      possibilities = metadata_dict[dataset][attribute]
      possibilities += [f"{elem}s" if elem[-1] != "y" else f"{elem[:-1]}ies" for elem in possibilities]

      evaluation[prompt][dataset][attribute] = {}
      evaluation[prompt][dataset][attribute]["total_hits"] = 0

      for rep in results[dataset][attribute]:
          
          if any(elem in re.split('[^a-zA-Z]', results[dataset][attribute][rep][prompt].lower()) for elem in possibilities):
            evaluation[prompt][dataset][attribute][f"hit_{rep}"] = 1
            evaluation[prompt][dataset][attribute]["total_hits"] += 1
          else:
            evaluation[prompt][dataset][attribute][f"hit_{rep}"] = 0

      evaluation[prompt][dataset][attribute]["average_hits"] = evaluation[prompt][dataset][attribute]["total_hits"] / n
      evaluation[prompt][dataset]["total_hits"] += evaluation[prompt][dataset][attribute]["average_hits"]
      print(f"Attribute {attribute}: average_hits = {evaluation[prompt][dataset][attribute]['average_hits']}")

    evaluation[prompt][dataset]["average_hits"] = evaluation[prompt][dataset]["total_hits"] / len(results[dataset]) # num of attributes
    print(f"\nDataset average hits {prompt}: {evaluation[prompt][dataset]['average_hits']}\n\n")
  

 


prompt_1
Dataset: City_Zhvi_AllHomes.csv

Attribute col2: average_hits = 0.4
Attribute col3: average_hits = 0.8
Attribute col4: average_hits = 0.0
Attribute col5: average_hits = 0.6

Dataset average hits prompt_1: 0.45000000000000007


Dataset: KaggleV2-May-2016.csv

Attribute col3: average_hits = 0.2
Attribute col4: average_hits = 1.0
Attribute col5: average_hits = 1.0
Attribute col6: average_hits = 0.0
Attribute col7: average_hits = 0.0

Dataset average hits prompt_1: 0.44000000000000006


Dataset: countries of the world.csv

Attribute col1: average_hits = 0.2
Attribute col2: average_hits = 0.0

Dataset average hits prompt_1: 0.1


Dataset: name_gender_dataset.csv

Attribute col1: average_hits = 0.0
Attribute col2: average_hits = 0.0

Dataset average hits prompt_1: 0.0


Dataset: speeddating.csv

Attribute col3: average_hits = 0.2
Attribute col4: average_hits = 0.0
Attribute col8: average_hits = 0.0
Attribute col9: average_hits = 0.0

Dataset average hits prompt_1: 0.05


prompt_2
Da